### Libraries Import

In [ ]:
import os
import glob
import shutil
import time
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import tensorflow.keras.backend as K
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Lambda
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image as kimage
from keras.applications.vgg16 import preprocess_input
import scipy.spatial.distance
import pickle
from scipy import interpolate
from collections import Counter
from pathlib import Path
%matplotlib inline

### Dataset import

#### Function to prepare the dataset structure:

In [ ]:
def read_image(img_path):
    img = kimage.load_img(img_path)
    x = kimage.img_to_array(img)
    return x

In [ ]:
consolidated_jpgdir = '/mnt/data/style_transfers/datasets/consolidated/jpg/'
dataset_folder = "./images_c/"
train_traversals = ['A000', 'A001']
test_traversals = ['A003', 'A058', 'A059', 'A064', 'A155', 'A159', 'A035', 'A037', 'A079', 'A086', 'A150', 'A151', 'A025', 'A026', 'A062', 'A063', 'A064', 'A138', 'A007']
test_traversals = sorted(test_traversals)
all_traversals = [str(i) for i in range(1,179,1)]
all_traversals = ['A'+(len(str(179))-len(digit))*'0'+digit for digit in all_traversals]
test_filenames = np.concatenate([glob.glob(consolidated_jpgdir + i + '*..jpg') for i in test_traversals]).flat
train_filenames = np.concatenate([glob.glob(consolidated_jpgdir + i + '*..jpg') for i in train_traversals]).flat
test_filenames = np.concatenate([glob.glob(consolidated_jpgdir + i + '*..jpg') for i in test_traversals]).flat
train_filenames = sorted(train_filenames)
test_filenames = sorted(test_filenames)

# in case we want to rebuild the structure, delete all the directories in './images'
if not os.path.exists(dataset_folder + 'training'):
    os.makedirs(dataset_folder + 'training')
    for f in train_filenames:
        shutil.copy(f, dataset_folder + 'training/.')
if not os.path.exists(dataset_folder + 'testing'):
    os.makedirs(dataset_folder + 'testing')
    for f in test_filenames:
        shutil.copy(f, dataset_folder + 'testing/.')

img = read_image(dataset_folder + 'training/A000_000.000..jpg')
(H, W, D) = img.shape
print("[INFO] {} train images and {} test images splited into directories.".format(len(train_traversals), len(test_traversals), len(all_traversals)))

#### Loading the images into array (in memory):

In [ ]:
start = time.time()
images_train = np.zeros((len(train_filenames), H, W, D), dtype=np.uint8)
images_test = np.zeros((len(test_filenames), H, W, D), dtype=np.uint8)
for i, filename in enumerate(train_filenames):
    images_train[i,:] = read_image(filename)
for i, filename in enumerate(test_filenames):
    images_test[i,:] = read_image(filename)
print ('[INFO] {} images loaded in {:.1f} seconds.'.format(len(images_train)+len(images_test), (time.time()-start)))

#### Getting the horizontal offset from ground truth

In [ ]:
consolidated_gtdir = '/mnt/data/style_transfers/datasets/consolidated/gt/'
def get_ground_truth(traversal_a, traversal_b):
    gt = np.zeros(32, dtype=int)
    gt_filenames = []
    gt_filenames.append(consolidated_gtdir + traversal_a + '_GT.txt')
    gt_filenames.append(consolidated_gtdir + traversal_b + '_GT.txt')
    gt_filenames = sorted(gt_filenames)
    gts = np.zeros((2,32), dtype=int)
    gt = []
    for i, filename in enumerate(gt_filenames):
        data = []
        with open(filename, "r") as f:
            data = f.read()
        data = data.split("\n")
        data = list(filter(None, data))
        horizontal = [int(i.split(" ")[0]) for i in data]
        gts[i,:] = horizontal 
    for i in range(32):
        gt.append(gts[1,i] - gts[0,i])
    print("[INFO] Ground truth loaded.")
    return np.array(gt)

In [ ]:
gt_train = get_ground_truth('A000', 'A001')

In [ ]:
gt_ = list(filter(lambda x: x < 500 and x > -500, gt_train))
gt_mean = np.mean([np.mean(gt__) for gt__ in gt_])
gt_median = np.median([np.median(gt__) for gt__ in gt_])
gt_min = np.min([np.min(gt__) for gt__ in gt_])
gt_max = np.max([np.max(gt__) for gt__ in gt_])

print('MEAN: ', gt_mean, '\nMEDIAN:', gt_median, "\nMIN:", gt_min, "\nMAX:", gt_max)

### Sliding the image 

In [ ]:
def cv2_imshow(a, title=None, mode=None, i=None, j=None, k=None, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    if mode == 'subplot':
        plt.subplot(i, j, k)
        return plt.imshow(a, **kwargs)
    else:
        plt.figure(figsize=(12,8))
        plt.title(title)
        return plt.imshow(a, **kwargs)
def imshow(a, title=None, mode=None, i=None, j=None, k=None, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if mode == 'subplot':
        plt.subplot(i, j, k)
        plt.title(title)
        return plt.imshow(a, **kwargs)
    else:
        plt.figure(figsize=(12,8))
        plt.tight_layout()
        plt.title(title)
        plt.imshow(a, **kwargs)
        return plt.show()
def process_image(img):
    x = cv2.resize(img, (224,224))
    return np.array(x, dtype=float)
def get_slices(img, offset_size, slides_count):
    a = np.zeros((slides_count, 224, 224, 3))
    for i in range(0, slides_count):
        a[i,:] = process_image(img[:, offset_size*i:offset_size*i+H])
    return a

## 1. Generating pairs

In [ ]:
def make_pairs(base_images, target_images, gt, offset_size, threshold):
    pairImages = []
    pairLabels = []
    slides_count = (W - H) // offset_size
    
    # loop over all images
    for seq_idx in range(32):
        target_gt = gt[seq_idx]
        base_image = base_images[seq_idx]
        target_image = target_images[seq_idx]
        base_slices = np.array([process_image(bimg) for bimg in [base_image[:,:H], base_image[:,(W-H)//2:-(W-H)//2], base_image[:,-H:]]])
        slides = get_slices(target_image, offset_size, slides_count)    # already processed
        for j, target_slide in enumerate(slides):
            pairImages.append([base_slices[0], target_slide])  # LEFT
            pairImages.append([base_slices[1], target_slide])  # MIDDLE
            pairImages.append([base_slices[2], target_slide])  # RIGHT
            if abs(target_gt - j*offset_size) < threshold:     # LEFT
                pairLabels.append([1])
            else:
                pairLabels.append([0])
            if abs((W-H)//2+target_gt - j*offset_size) < threshold: # MIDDLE
                pairLabels.append([1])
            else:
                pairLabels.append([0])
            if abs((W-H)+target_gt - j*offset_size) < threshold: # RIGHT
                pairLabels.append([1])
            else:
                pairLabels.append([0])
    return (np.array(pairImages), np.array(pairLabels))

## 2. Building the Siamese NN

### Pre-trained VGG16 'sister' network architecture:

In [ ]:
def buildVGG16_model(input_shape):
    vgg16 = keras.applications.VGG16(weights='imagenet', include_top=True, pooling='max', input_shape=input_shape)
    basemodel = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
    return basemodel

In [ ]:
def get_feature_vector(basemodel, img):
    feature_vector = basemodel.predict(img)
    return feature_vector
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))
def contrastive_loss(y_true, y_pred, margin=1.0):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    return y_true * tf.math.square(y_pred) + (1.0 - y_true) * tf.math.square(tf.math.maximum(margin - y_pred, 0.0))

## 3. Training the network

In [ ]:
def plot_training(H, plotPath, mode='normal'):
    if mode == 'loaded':
        # construct a plot that plots and saves the training history
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(H["loss"], label="train_loss")
        plt.plot(H["accuracy"], label="train_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.savefig(plotPath)
    else:
        # construct a plot that plots and saves the training history
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(H.history["loss"], label="train_loss")
        plt.plot(H.history["accuracy"], label="train_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.savefig(plotPath)

#### Setting the parameters:

In [ ]:
offset_size = 5
threshold = offset_size * 1.3
batch_size = 64
epochs = 5
IMG_SHAPE = (224,224,3)

BASE_OUTPUT = "output5pxc"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

In [ ]:
start = time.time()
(pairTrain, labelsTrain) = make_pairs(images_train[:32], images_train[32:], gt_train, offset_size, threshold)
print ('[INFO] {} pairs of image paths with offsets were generated in {:.1f} seconds.'.format(len(pairTrain), (time.time()-start)))

### Training the Siamese network

In [ ]:
print("[INFO] building siamese network...")
IMG_SHAPE = (224,224,3)
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
VGGfeatureExtractor = buildVGG16_model(IMG_SHAPE) # building the network only once since we want to share the weights 
featsA = VGGfeatureExtractor(imgA)
featsB = VGGfeatureExtractor(imgB)

In [ ]:
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)
print(model.summary())

In [ ]:
checkpoint_path = BASE_OUTPUT + "/training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#model.load_weights(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

def merge_dicts(d1, d2):
    d = {}
    for key in d1.keys():
        l1 = d1[key]
        l2 = d2[key]
        d[key] = l1 + l2
    return d

In [ ]:
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
print("[INFO] training model...")
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelsTrain[:],
    batch_size=batch_size,
    verbose=1,
    epochs=epochs,
    callbacks=[cp_callback])


# serialize model to JSON
model_json = model.to_json()
with open(MODEL_PATH + "/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_PATH + "/model_weights.h5")
print("Saved model to disk")

# plot the training history
print("[INFO] plotting current training history...")
plot_training(history, PLOT_PATH)

# merge with existing training and save it
loaded_history = pickle.load(open(BASE_OUTPUT + '/trainHistoryDict', "rb"))
new_history = merge_dicts(loaded_history, history.history)
print("[INFO] plotting the whole training history...")
plot_training(new_history, PLOT_PATH, "loaded")

with open(BASE_OUTPUT + '/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(new_history, file_pi)

In [ ]:
# load json and create model
json_file = open(MODEL_PATH + '/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(MODEL_PATH + "/model_weights.h5")
print("Loaded model from disk")
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## 4. Testing the network

In [ ]:
from collections import Counter
def get_offsets(model, base_images, target_images, base_part, slides_count, offset_size):
    sims = np.zeros((len(base_images), slides_count, 1))
    for i in range(len(base_images)):
        base_image = base_images[i]
        target_image = target_images[i]
        base_slices = np.array([process_image(bimg) for bimg in [base_image[:,:H], base_image[:,(W-H)//2:-(W-H)//2], base_image[:,-H:]]], dtype=float)
        target_slices = get_slices(target_image, offset_size, slides_count)
        base_slices_ = np.array([base_slices[base_part] for _ in range(slides_count)])
        pair = [base_slices_, target_slices]
        similarity = model.predict(pair)
        sims[i,:] = similarity
    return np.array(sims)
def get_offsets_(model, base_slices, target_images, base_part, slides_count, offset_size):
    sims = np.zeros((len(target_images), slides_count, 1))
    for i in range(len(target_images)):
        target_image = target_images[i]
        target_slices = get_slices(target_image, offset_size, slides_count)
        base_slices_ = np.array([base_slices[base_part] for _ in range(slides_count)])
        pair = [base_slices_, target_slices]
        similarity = model.predict(pair)
        sims[i,:] = similarity
    return np.array(sims)
def closest_number(n, m) : 
    q = int(n / m) 
    n1 = m * q 
    if((n * m) > 0) : 
        n2 = (m * (q + 1))  
    else : 
        n2 = (m * (q - 1)) 
    if (abs(n - n1) < abs(n - n2)) : 
        return n1       
    return n2 
def show_offsets(base_images, target_images, similarities, gt, traversal_names_a, traversal_names_b, base_part, offset_size, top_n):
    gt_offsets = [0, (W-H)//2,(W-H)]
    line_separator = "---------------------------------------------------------------------------------------------------------------------\n"
    parts = ["LEFT", "CENTER", "RIGHT"]
    if(base_part < 3):
        print("PREDICTIONS FOR THE " + parts[base_part] + " PART OF THE BASE IMAGE:\n\n")
        for i, similarity in enumerate(similarities):
            print(line_separator + "Test image ", traversal_names[i], "\n")
            print(gt[i] + gt_offsets[base_part], "    true offset (in px)")
            print(np.argmax(similarity) * offset_size, '     predicted offset (in px)\n')
    else:
        print("PREDICTIONS FOR ALL PARTS OF THE BASE IMAGE:\n\n")
        space_separator = "                    |                    "
        small_separator = "     |     "
        tiny_separator = "  |  "
        print(parts[0] + space_separator + parts[1] + space_separator + parts[2] + "\n" + line_separator)
        for i in range(len(similarities)):
            base_image = base_images[i]
            target_image = target_images[i]
            print(line_separator + "Pair", traversal_names_a[i][-17:-5], ":", traversal_names_b[i][-17:-5], "\n")
            plt.figure(figsize=(8,4))
            plt.tight_layout()
            imshow(base_image, 'base image', 'subplot', 1, 2, 1)
            imshow(target_image, 'target image', 'subplot', 1, 2, 2)
            plt.show()
            print("True offset:     " + str(gt[i])) 
            left = similarities[i,0,:]
            center = similarities[i,1,:]
            right = similarities[i,2,:]
            sorted_left = (-left).argsort()[:top_n] * offset_size - gt_offsets[0]
            sorted_center = (-center).argsort()[:top_n] * offset_size - gt_offsets[1]
            sorted_right = (-right).argsort()[:top_n] * offset_size - gt_offsets[2]
            rounded_left = [closest_number(n, offset_size) for n in sorted_left]
            rounded_center = [closest_number(n, offset_size) for n in sorted_center]
            rounded_right = [closest_number(n, offset_size) for n in sorted_right]
            print("Predicted offsets:")
            print(parts[0] + space_separator + parts[1] + small_separator + parts[2] + "\n" + line_separator)
            print(str(rounded_left) + small_separator + "  " + 
                  str(rounded_center) + small_separator +
                  str(rounded_right))
            counter = Counter(rounded_left + rounded_center + rounded_right)
            (val, cnt) = counter.most_common(1)[0]
            print("Most frequent offset: ", val)
def process_offsets(similarities, offset_size, sliced_count):
    gt_offsets = [0, (W-H)//2,(W-H)]
    offsets = np.zeros((len(similarities), len(similarities[0]), slides_count)) # 3 x 32 x 'slides_count'
    # Iterate throught target images...
    for i in range(len(similarities)):
        left = similarities[i,0,:]
        center = similarities[i,1,:]
        right = similarities[i,2,:]
        sorted_left = (-left).argsort() * offset_size - gt_offsets[0]
        sorted_center = (-center).argsort() * offset_size - gt_offsets[1]
        sorted_right = (-right).argsort() * offset_size - gt_offsets[2]
        rounded_left = [closest_number(n, offset_size) for n in sorted_left]
        rounded_center = [closest_number(n, offset_size) for n in sorted_center]
        rounded_right = [closest_number(n, offset_size) for n in sorted_right]  
        offsets[i,0,:] = sorted_left
        offsets[i,1,:] = rounded_center
        offsets[i,2,:] = rounded_right
    return offsets
            
def save_offsets(offsets, filename):
    with open(BASE_OUTPUT + "/" + filename, 'wb') as file_pi:
        pickle.dump(offsets, file_pi)
    print("[INFO] Offsets pickled into file '" + filename + "'.")
def load_offsets(filepath):
    return pickle.load(open(filepath, "rb")) 

In [ ]:
start = time.time()
slides_count = (W - H) // offset_size
fst_traversal = 'A000'
snd_traversal = 'A001'

gt_test = get_ground_truth(fst_traversal, snd_traversal)
center_offsets = get_offsets(model, images_train[:32], images_train[32:], 1, slides_count, offset_size)

print ('[INFO] For 1 position, the offsets were predicted in {:.1f} seconds.'.format((time.time()-start)))

# 5. Testing Framework

1. load images from 'positioned' directory
2. take the first as a base image and compute the predicted offsets for all images
3. save the offset into 'results' directory

In [ ]:
# resultsDir = "./results/"
offset_size = 5
model_type_flag = "cspline"
first_time = True
slides_count = (W-H) // offset_size
fst_traversal = 'A000'
spc = "   |   "
big_spc = "         |        "

# load all offsets
traversals = os.listdir("./positioned/1/")
traversals = [f[:4] for f in traversals]
traversals = sorted(traversals)
center_bins = np.array([i for i in range(slides_count)]) * offset_size - (W-H)//2

sims_idx = np.array([i if abs(b) > 30 else -1  for i, b in enumerate(center_bins)])
sims_idx = np.where(sims_idx != -1)

if(traversals[0] != fst_traversal):
    print("[ERROR] The base image A000 has to be at the first place!")
    
offsets = np.zeros((len(traversals)-1, 32, slides_count, 1))
target_traversals = traversals[1:]
for i, snd_traversal in enumerate(target_traversals):
    offsetsDir = BASE_OUTPUT + '/offsets/' + fst_traversal + '-' + snd_traversal
    current_traversal_offsets = load_offsets(offsetsDir)
    offsets[i,:] = current_traversal_offsets
gts = np.zeros((len(target_traversals), 32))
for i in range(len(gts)):
    gts[i] = get_ground_truth(fst_traversal, target_traversals[i])
    
#get the peak of similarities for each of the position and each of the traversals, then save it into 'results' dir
for files in os.walk("./positioned/"):
    start = time.time()
    if files[0] == "./positioned/":
        continue
    
    position = files[0].split("/")[-1]
    print("==============")
    print("Position: " + str(position))
    print("==============")
    filenames = files[2]  # A000_000.000-xx.bmp etc.
    filenames = sorted(filenames)
    
    # Build results directories if not done yet
    if first_time:
        try:
            for j in range(1, 33):
                Path(resultsDir + "snn_" + str(offset_size) + "px" + model_type_flag + "/" + str(j)).mkdir(parents=True, exist_ok=True)
        except Exception as e:
            print(e)
        first_time = False
        
    # Get the most promising offset from these and save it into results directory
    print("Predicted offsets:")
    print("POSITION" + spc + "TRAVERSAL" + spc + "PEAK" + big_spc + "PEAK VAL" + spc + "SPLINE PEAK" + spc + "SPLINE PEAK VAL" + spc + "GT")
    for i, traversal_offsets in enumerate(offsets):
        similarities = traversal_offsets[int(position)-1,:,0]
        # 1. get the histogram RAW data peak
        similarities[sims_idx] = 0
        peak_idx = np.argmax(similarities)
        peak = center_bins[peak_idx]
        peak_val = np.max(similarities)
        
        # 2. get the spline peak
        tck,_     = interpolate.splprep( [center_bins, similarities] ,s = 10)
        spline_bins,spline = interpolate.splev( np.linspace( 0, 1, 1000), tck, der = 0)
        spline[sims_idx] = -1
        spline_peak_idx = np.argmax(spline)
        spline_peak = int(spline_bins[spline_peak_idx])
        spline_peak_val = np.max(spline)
        
        print(str(position) + big_spc + str(target_traversals[i]) + spc + str(peak) + big_spc + str(peak_val)[:5] + spc + str(spline_peak) + big_spc + str(spline_peak_val)[:5] + big_spc + str(gts[i,int(position)-1]))
        fn = os.path.join(resultsDir + "snn_" + str(offset_size) + "px" + model_type_flag + "/" + str(position), filenames[i+1] + ".best")
        with open(fn, "w") as f:
            f.write(str(spline_peak) + "\n")
    print ('[INFO] On position #{}, the offsets were predicted in {:.1f} seconds.'.format(position, (time.time()-start)))

# 6. Interpret the results

1. Calculate a histogram from similarities with center part of the base image
2. Plot it with and without using spline function

In [ ]:
start = time.time()
slides_count = (W - H) // offset_size
fst_traversal = 'A000'
snd_traversal = 'A005'

gt_test = get_ground_truth(fst_traversal, snd_traversal)
center_offsets = get_offsets(model, images_train[:1*32], images_test[3*32:4*32], 1, slides_count, offset_size)

offsetsDir =  '/offsets/' + fst_traversal + '-' + snd_traversal
save_offsets(center_offsets, offsetsDir)
center_bins = np.array([i for i in range(slides_count)]) * offset_size - (W-H)//2
print ('[INFO] For 1 position, the offsets were predicted in {:.1f} min.'.format((time.time()-start) // 60))

Compute offsets for all selected traversals:

### Load offsets from a file and save the plot to a file:
* get number of base traversal and target traversal as an input
* load offsets from these traversals
* plot the offsets for all positions and save the images with plot to a file

In [ ]:
test_traversals

In [ ]:
slides_count = (W - H) // offset_size
fst_traversal = 'A000'
snd_traversal = 'A059'
plotDir = BASE_OUTPUT + '/plots/' + fst_traversal + '-' + snd_traversal + '/'
offsetsDir = BASE_OUTPUT + '/offsets/' + fst_traversal + '-' + snd_traversal
Path(plotDir).mkdir(parents=True, exist_ok=True)

center_offsets = load_offsets(offsetsDir)
center_bins = np.array([i for i in range(slides_count)]) * offset_size - (W-H)//2
gt_test = get_ground_truth(fst_traversal, snd_traversal)

fst_traversal_images = images_train[:32]
fst_traversal_names = train_filenames[:32]
snd_index = test_traversals.index(snd_traversal)
snd_traversal_images = images_test[snd_index*32:(snd_index+1)*32]
snd_traversal_names = test_filenames[snd_index*32:(snd_index+1)*32]

for image_index in range(0,32,1):
    similarity = center_offsets[image_index,:,0]
    tck,u     = interpolate.splprep( [center_bins, similarity] ,s = 10)
    xnew,ynew = interpolate.splev( np.linspace( 0, 1, 1000), tck, der = 0)
    true_offset = gt_test[image_index]
    rounded_offset = closest_number(true_offset, offset_size) - 1
    if(rounded_offset > center_bins[-1] or rounded_offset < center_bins[0]):
        print('The offset {} is too big/small for the trained network.'.format(rounded_offset))
        continue
    plt.figure(figsize=(16,8))
    imshow(fst_traversal_images[image_index], 'Image ' + fst_traversal_names[image_index][-17:-5], 'subplot', 1,3,1)
    imshow(snd_traversal_images[image_index], 'Image ' + snd_traversal_names[image_index][-17:-5], 'subplot', 1,3,2)
    plt.subplot(1,3,3)
    plt.plot(center_bins, similarity, color='r')
    plt.plot(xnew, ynew, color='b')
    plt.plot(rounded_offset, similarity[np.where(center_bins == rounded_offset)], 'o', color='black')
    plt.legend( [ 'data' , 'spline', 'GT'] )
    plt.axvline(x=rounded_offset, color='black')
    plt.title('GT is {} px offset, {} px offset has similarity {:.2f}'.format(true_offset, rounded_offset, similarity[np.where(center_bins == rounded_offset)][0]))
    plt.grid()
    plotPath = plotDir + train_filenames[image_index][-12:-5] + ".png"
    plt.savefig(plotPath)

In [ ]:
slides_count = (W - H) // offset_size
center_bins = np.array([i for i in range(slides_count)]) * offset_size - (W-H)//2
fst_traversal = 'A000'
for snd_traversal in test_traversals:
    plotDir = BASE_OUTPUT + '/plots/' + fst_traversal + '-' + snd_traversal + '/'
    offsetsDir = BASE_OUTPUT + '/offsets/' + fst_traversal + '-' + snd_traversal
    Path(plotDir).mkdir(parents=True, exist_ok=True)

    center_offsets = load_offsets(offsetsDir)
    gt_test = get_ground_truth(fst_traversal, snd_traversal)

    fst_traversal_images = images_train[:32]
    fst_traversal_names = train_filenames[:32]
    snd_index = test_traversals.index(snd_traversal)
    snd_traversal_images = images_test[snd_index*32:(snd_index+1)*32]
    snd_traversal_names = test_filenames[snd_index*32:(snd_index+1)*32]

    for image_index in range(0,32,1):
        similarity = center_offsets[image_index,:,0]
        tck,u     = interpolate.splprep( [center_bins, similarity] ,s = 0.01)
        xnew,ynew = interpolate.splev( np.linspace( 0, 1, 200), tck, der = 0)
        true_offset = gt_test[image_index]
        rounded_offset = closest_number(true_offset, offset_size) - 1
        if(rounded_offset > center_bins[-1] or rounded_offset < center_bins[0]):
            print('The offset {} is too big/small for the trained network.'.format(rounded_offset))
            continue
        plt.figure(figsize=(16,8))
        imshow(fst_traversal_images[image_index], 'Image ' + fst_traversal_names[image_index][-17:-5], 'subplot', 1,3,1)
        imshow(snd_traversal_images[image_index], 'Image ' + snd_traversal_names[image_index][-17:-5], 'subplot', 1,3,2)
        plt.subplot(1,3,3)
        plt.plot(center_bins, similarity, color='r')
        plt.plot(xnew, ynew, color='b')
        plt.plot(rounded_offset, similarity[np.where(center_bins == rounded_offset)], 'o', color='black')
        plt.legend( [ 'data' , 'spline', 'GT'] )
        plt.axvline(x=rounded_offset, color='black')
        plt.title('GT is {} px offset, {} px offset has similarity {:.2f}'.format(true_offset, rounded_offset, similarity[np.where(center_bins == rounded_offset)][0]))
        plt.grid()
        plotPath = plotDir + train_filenames[image_index][-12:-5] + ".png"
        plt.savefig(plotPath)

Show plots from file:

In [ ]:
slides_count = (W - H) // offset_size
center_bins = np.array([i for i in range(slides_count)]) * offset_size - (W-H)//2
fst_traversal = 'A000'
snd_traversal = 'A059'
plotDir = BASE_OUTPUT + '/plots/' + fst_traversal + '-' + snd_traversal + '/'
plotPath = plotDir + train_filenames[image_index][-12:-5] + ".png"
plot = read_image(plotPath)
cv2.imshow('title', plot)
imshow(plot)